In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir, listdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
from datetime import date
import pandas as pd
import numpy as np

In [ ]:
from bulletin import default_output, tables_path
from bulletin.systems.notifica import Notifica

In [ ]:
tables = {f:pd.read_csv(join(tables_path,f'{f}.csv')) for f in [ Path(f).stem for f in listdir(tables_path) if '.csv' in f ] }
tables['faixa_etaria_populacao']

In [ ]:
nt = Notifica()
nt.load('notifica', compress=False)
nt.df

In [ ]:
nt.tables['instituicao_ensino'] = nt.tables['instituicao_ensino'].set_index('id')
nt.tables['instituicao_ensino'].loc[-1] = 'Não Informado'

In [ ]:
total_periodo = nt.df.loc[(nt.df['data_notificacao']>='2022-02-14') & (nt.df['data_notificacao']<='2022-02-18'),['id','idade','classificacao_final','evolucao','data_diagnostico','instituicao_ensino','excluir_ficha','status_notificacao']].join(nt.tables['instituicao_ensino'], on='instituicao_ensino')

In [ ]:
total_periodo.groupby(['dependencia_administrativa','classificacao_final'],dropna=False)[['id']].count().unstack('classificacao_final').rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_values(by='Caso confirmado')

In [ ]:
total_periodo.pivot_table(index='dependencia_administrativa', columns='classificacao_final', values='id', aggfunc='count', margins=True, margins_name='Total',fill_value=0).rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final']).sort_values(by='Total')

In [ ]:
total_periodo['idade_bins'] = np.digitize(total_periodo['idade'],[10,17],right=True)
total_periodo.pivot_table(index='dependencia_administrativa', columns='idade_bins', values='id', aggfunc='count', margins=True, margins_name='Total',fill_value=0).rename(index=nt.tables['classificacao_final'].set_index('id')['classificacao_final']).sort_values(by='Total').rename(columns={0:'0-10',1:'11-17',2:'18+'}, level='idade_bins')

In [ ]:
possiveis_estudantes = nt.df.loc[(nt.df['idade']>=11) & (nt.df['idade']<=17) & (nt.df['data_notificacao']>='2022-02-14') & (nt.df['data_notificacao']<='2022-02-18'),['id','idade','classificacao_final','evolucao','data_diagnostico','instituicao_ensino','excluir_ficha','status_notificacao']].join(nt.tables['instituicao_ensino'], on='instituicao_ensino')

In [ ]:
possiveis_estudantes.groupby(['dependencia_administrativa','classificacao_final'],dropna=False)[['id']].count().unstack('classificacao_final').rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final']).fillna(0).astype(int).droplevel(axis=1,level=0).sort_values(by='Caso confirmado')

In [ ]:
possiveis_estudantes.pivot_table(index='dependencia_administrativa', columns='classificacao_final', values='id', aggfunc='count', margins=True, margins_name='Total',fill_value=0).rename(columns=nt.tables['classificacao_final'].set_index('id')['classificacao_final']).sort_values(by='Total')